In [ ]:
import pandas as pd
import plotly.express as ply
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels import tsa
import numpy as np
import statistics
import plotly.express as px
import seaborn as sns

In [ ]:
import datetime as datetime

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
pip install pmdarima

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from math import  sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
#pip install statsmodels --upgrade
!pip install statsmodels
!pip3 install --user scipy==1.2.0

In [ ]:
from pmdarima import auto_arima

In [ ]:
df=pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
df.head(2)

In [ ]:
df.tail(3)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Date']=pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [ ]:
df.set_index('Date',inplace=True)

**Analyzing the raise of cases on monthly basis**

To analyze the confirmed cases on monthly basis we need to resample the data on monthly basis.




In [ ]:
df['Confirmed'].resample(rule='M').sum().plot(figsize=(25,5))

After resmapling the data month wise we were able analyze the increase of covid 19 confirmed cases after the July 2020.

**Analyzing the confirmed cases on Quaterly basis**

In [ ]:
df['Confirmed'].resample(rule='Q').sum().plot(figsize=(25,5))
plt.xlabel('Date')
plt.ylabel('count of Confirmed cases')
plt.show()

There is raise in covid 19 cases after third quater in 2020 and there is a huge raise after the first quater 2021. 

In [ ]:
df1=pd.DataFrame()
df1=df.drop(columns=['Sno','Time','ConfirmedIndianNational','State/UnionTerritory','ConfirmedForeignNational','Cured','Deaths'])

In [ ]:
df1['Confirmed'].plot(color='red')
plt.title('Confirmed cases to analyze the trend')
plt.show()

**Dataset  consists of repeated number of tupes on a same date in different states. Our main objective is to analyze the confirmed cases in India so we will try to perform resampling on day wise sum (total number of confirmed cases)**

In [ ]:
df.loc['2020-12-03']

In [ ]:
df_day=df1.resample('D').sum()

**After resampling we will be able to the total number of confirmed cases in India**

In [ ]:
df_day.loc['2020-12-03']

**Seasonal Decomposition**

By using seasonal decomposition we will be able to find the trend, Seasonal and Residual error components in data.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df_day['Confirmed'])
result.plot()
plt.show()

After plotting seasonal decomposition plot we will be knowing data consist of trend component in it.


**Checking Stationarity**

Test Stationarity function helps us to stationarity in data by find the mean, standard deviation are constant or not over time. We will also adfuller test to check stationarity by using p-value if p-value is less 5% then it statisfies the null hypothesis and stationarity is present in data.

---



In [ ]:
def test_stationarity(timeseries):
    
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
              dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(df_day['Confirmed'])

In [ ]:
df_day['diff1']=df_day['Confirmed']-df_day['Confirmed'].shift(1)
test_stationarity(df_day['diff1'].dropna())


In [ ]:
df_day['diff2']=df_day['Confirmed']-df_day['Confirmed'].shift(2)
test_stationarity(df_day['diff2'].dropna())

**Plotting acf and pacf**

**The acf helps us to identify the approximate values of moving average term used for the model.**

In [ ]:
plot_acf(df_day['diff2'].dropna(),lags=15)
plt.show()

**Pacf plot helps us to identify the approximate value of auto regressive model term used for the model. In some cases it can also be used to find the seasonality.**

In [ ]:
plot_pacf(df_day['diff2'].dropna(),lags=15)
plt.show()

**Fitting Approximate models**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
#Model 1 
arima_model_121 = ARIMA(df_day['Confirmed'].dropna(), order=(1,2,1)).fit()

In [ ]:
arima_model_121.summary()


In [ ]:
start = df_day.index.get_loc('2020-10-6')
end = df_day.index.get_loc('2021-05-28')
print(start,'to',end)

In [ ]:
 predict_121= arima_model_121.predict(start=start+1 ,end=end+1, dynamic= False)  
    

**Forcasting model for next 300 days i.e, approximately for next 8 months.**


In [ ]:
#Trying to forecast future values
pred_future_121=arima_model_121.predict(start=len(df_day)+1,end=len(df_day)+(300),dynamic=False)
print("The length of pred_future values :",len(pred_future_121))

In [ ]:
plt.plot(df_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_121,color='green',label='Testing')
plt.plot(pred_future_121,color='red',label='pink')
plt.xticks(rotation=45)
plt.legend()
plt.show()

**Traing & Testing  the model**

In [ ]:
train=df_day[:len(df_day)-(89)]
test=df_day[len(df_day)-89:]

In [ ]:
 predict_121= arima_model_121.predict(start=len(train) ,end=len(df_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_121))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_121))
r2=r2_score(test['Confirmed'],predict_121)
mse=mean_squared_error(test['Confirmed'],predict_121)
mae=mean_absolute_error(test['Confirmed'],predict_121)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

In [ ]:
#Model 2
arima_model_221 = ARIMA(df_day['Confirmed'].dropna(), order=(2,2,1)).fit()


In [ ]:
arima_model_221.summary()

In [ ]:
start = df_day.index.get_loc('2020-10-6')
end = df_day.index.get_loc('2021-05-28')
print(start,'to',end)

In [ ]:
 predict_221= arima_model_221.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_221=arima_model_221.predict(start=len(df_day)+1,end=len(df_day)+(300),dynamic=False)
print("The length of pred_future values :",len(pred_future_221))


In [ ]:
plt.plot(df_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_221,color='green',label='Testing')
plt.plot(pred_future_221,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
 predict_221= arima_model_221.predict(start=len(train) ,end=len(df_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_221))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_221))
r2=r2_score(test['Confirmed'],predict_221)
mse=mean_squared_error(test['Confirmed'],predict_221)
mae=mean_absolute_error(test['Confirmed'],predict_221)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

In [ ]:
# Identifying Best Suitable model 
auto_arima_model=auto_arima(df_day['Confirmed'],trace=True,Supress_warnings=True)

In [ ]:
#Best Model 
arima_model_122 = ARIMA(df_day['Confirmed'].dropna(), order=(1,2,2)).fit()

In [ ]:
arima_model_122.summary()

In [ ]:
start = df_day.index.get_loc('2020-10-6')
end = df_day.index.get_loc('2021-05-28')
print(start,'to',end)

In [ ]:
 predict_122= arima_model_122.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_122=arima_model_122.predict(start=len(df_day)+1,end=len(df_day)+(200),dynamic=False)
print("The length of pred_future values :",len(pred_future_122))

In [ ]:
plt.plot(df_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_122,color='green',label='Testing')
plt.plot(pred_future_122,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
 predict_122= arima_model_122.predict(start=len(train) ,end=len(df_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
r2=r2_score(test['Confirmed'],predict_122)
mse=mean_squared_error(test['Confirmed'],predict_122)
mae=mean_absolute_error(test['Confirmed'],predict_122)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

**Forecasting the impact of Covid-19 in developed states in India**

Basing on the gdp per captia in there are 14 developed states.


They are
*  Maharashtra
*  Tamil Nadu
*  Gujarat
*  Karnataka
*  Uttar Pradesh
*  West Bengal
* Delhi
*  Rajasthan
*  Andhra Pradesh
* Telangana
* Kerala
* Haryana
* Madhya Pradesh
* Goa

So, initially we will be trying to study the impact of covid-19 cases in developed states of India. 

The list was taken based on the data avilable in the link given below.


https://www.innfinity.in/limitless/most-developed-state-in-india/

In [ ]:
df1=pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
df_developed_=df1.drop(columns=['ConfirmedIndianNational','ConfirmedForeignNational','Cured','Deaths','Time','Sno'])

In [ ]:
df_developed_.head(3)

In [ ]:
df_developed_.tail(3)

In [ ]:
Mh=df_developed_[df_developed_['State/UnionTerritory']=='Maharashtra']
Tn=df_developed_[df_developed_['State/UnionTerritory']=='Tamil Nadu']
Gj=df_developed_[df_developed_['State/UnionTerritory']=='Gujarat']
Mp=df_developed_[df_developed_['State/UnionTerritory']=='Madhya Pradesh']
Wb=df_developed_[df_developed_['State/UnionTerritory']=='West Bengal']
Rj=df_developed_[df_developed_['State/UnionTerritory']=='Rajasthan']
Ap=df_developed_[df_developed_['State/UnionTerritory']=='Andhra Pradesh']
Kn=df_developed_[df_developed_['State/UnionTerritory']=='Karnataka']
Ts=df_developed_[df_developed_['State/UnionTerritory']=='Telengana']
Kl=df_developed_[df_developed_['State/UnionTerritory']=='Kerala']
Hr=df_developed_[df_developed_['State/UnionTerritory']=='Haryana']
Dl=df_developed_[df_developed_['State/UnionTerritory']=='Delhi']
Ut=df_developed_[df_developed_['State/UnionTerritory']=='Uttar Pradesh']
Goa=df_developed_[df_developed_['State/UnionTerritory']=='Goa']

In [ ]:
df['State/UnionTerritory'].unique()

In [ ]:
with plt.xkcd():
  fig, ax=plt.subplots(nrows=4, ncols=3, figsize=(8,8), squeeze=False, sharex=True, sharey=False, constrained_layout=True )
  sns.kdeplot(data=Tn['Confirmed'],ax=ax[0,0],fill=True)
  ax[0,0].set_title('Tamil Nadu')
  sns.kdeplot(data=Mh['Confirmed'],ax=ax[0,1],fill=True)
  ax[0,1].set_title('Maharashtra')
  sns.kdeplot(data=Gj['Confirmed'],ax=ax[0,2],fill=True)
  ax[0,2].set_title('Gujarat')

  ax[1,0].set_title('Madhya pradesh')
  sns.kdeplot(data=Mp['Confirmed'],ax=ax[1,0],fill=True)
  ax[1,1].set_title('West Bengal')
  sns.kdeplot(data=Wb['Confirmed'],ax=ax[1,1],fill=True)
  ax[1,2].set_title('Rajasthan')
  sns.kdeplot(data=Rj['Confirmed'],ax=ax[1,2],fill=True)

  ax[2,0].set_title('Andhra Pradesh')
  sns.kdeplot(data=Ap['Confirmed'],ax=ax[2,0],fill=True)
  ax[2,1].set_title('Telengana')
  sns.kdeplot(data=Ts['Confirmed'],ax=ax[2,1],fill=True)
  ax[2,2].set_title('Karnataka')
  sns.kdeplot(data=Kn['Confirmed'],ax=ax[2,2],fill=True)

  
  ax[3,0].set_title('Kerala')
  sns.kdeplot(data=Kl['Confirmed'],ax=ax[3,0],fill=True)
  ax[3,1].set_title('Haryana')
  sns.kdeplot(data=Hr['Confirmed'],ax=ax[3,1],fill=True)
  ax[3,2].set_title('Delhi')
  sns.kdeplot(data=Dl['Confirmed'],ax=ax[3,2],fill=True)

  plt.show()

In [ ]:
states=['Maharashtra','Tamil Nadu','Gujarat','Madhya Pradesh','West Bengal','Rajasthan','Andhra Pradesh','Karnataka','Telengana','Kerala','Haryana','Delhi','Uttar Pradesh','Goa']

In [ ]:
df_developed_=df_developed_[df_developed_['State/UnionTerritory'].isin(states)]

In [ ]:
df_developed_['State/UnionTerritory'].unique()

In [ ]:
df_developed_.head(3)

In [ ]:
df_developed_.tail(3)

In [ ]:
len(df_developed_)

In [ ]:
df_developed_.drop(columns=['State/UnionTerritory'],inplace=True)

In [ ]:
df_developed_['Date']=pd.to_datetime(df_developed_['Date'], format='%Y-%m-%d')

In [ ]:
df_developed_.set_index('Date',inplace=True)

In [ ]:
df_developed_day=df_developed_.resample('D').sum()

**Seasonal Decomposition**

In [ ]:
result = seasonal_decompose(df_developed_day['Confirmed'])
result.plot()
plt.show()

In [ ]:
test_stationarity(df_developed_day['Confirmed'])

In [ ]:
df_developed_day['diff1']=df_developed_day['Confirmed']-df_developed_day['Confirmed'].shift(1)
test_stationarity(df_developed_day['diff1'].dropna())


In [ ]:
df_developed_day['diff2']=df_developed_day['Confirmed']-df_day['Confirmed'].shift(2)
test_stationarity(df_developed_day['diff2'].dropna())

In [ ]:
df_developed_day['diff3']=df_developed_day['Confirmed']-df_day['Confirmed'].shift(3)
test_stationarity(df_developed_day['diff3'].dropna())

In [ ]:
df_developed_day['log_c']= df_developed_day['Confirmed'].apply(lambda x: np.log(x))  
test_stationarity(df_developed_day.log_c)

**Plotting pacf and acf**

In [ ]:
plot_acf(df_developed_day['log_c'].dropna(),lags=15)
plt.show()

In [ ]:
plot_pacf(df_developed_day['log_c'],lags=15)
plt.show()

In [ ]:
train=df_developed_day[:len(df_day)-(89)]
test=df_developed_day[len(df_day)-89:]

In [ ]:
len(df_developed_day)

In [ ]:
#Model 1
arima_model_221 = ARIMA(df_developed_day['log_c'].dropna(), order=(2,2,1)).fit()

In [ ]:
arima_model_221.summary()

In [ ]:
start = df_day.index.get_loc('2021-05-13')
end = df_day.index.get_loc('2021-06-01')
print(start,'to',end)

In [ ]:
 predict_221= arima_model_221.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_221=arima_model_221.predict(start=len(df_developed_day)+1,end=len(df_developed_day)+(300),dynamic=False)
print("The length of pred_future values :",len(pred_future_221))

In [ ]:
plt.plot(df_developed_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_221,color='green',label='Testing')
plt.plot(pred_future_221,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
 predict_221= arima_model_221.predict(start=len(train) ,end=len(df_developed_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_221))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_221))
r2=r2_score(test['Confirmed'],predict_221)
mse=mean_squared_error(test['Confirmed'],predict_221)
mae=mean_absolute_error(test['Confirmed'],predict_221)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

In [ ]:
# Identifying Best Suitable model 
auto_arima_model=auto_arima(df_developed_day['log_c'],trace=True,Supress_warnings=True)

In [ ]:
#Best Model 
arima_model_321= ARIMA(df_developed_day['log_c'], order=(3,2,1)).fit()

In [ ]:
arima_model_321.summary()

In [ ]:
start = df_day.index.get_loc('2021-05-13')
end = df_day.index.get_loc('2021-06-01')
print(start,'to',end)

In [ ]:
 predict_321= arima_model_321.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_321=arima_model_321.predict(start=len(df_developed_day)+1,end=len(df_developed_day)+(250),dynamic=False)
print("The length of pred_future values :",len(pred_future_321))


In [ ]:
plt.plot(df_developed_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_321,color='yellow',label='Testing')
plt.plot(pred_future_321,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
 predict_321= arima_model_321.predict(start=len(train) ,end=len(df_developed_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_321))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_321))
r2=r2_score(test['Confirmed'],predict_321)
mse=mean_squared_error(test['Confirmed'],predict_321)
mae=mean_absolute_error(test['Confirmed'],predict_321)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

In [ ]:
# Identifying Best Suitable model 
auto_arima_model=auto_arima(df_developed_day['Confirmed'],trace=True,Supress_warnings=True)

In [ ]:
#Best Model 
arima_model_122 = ARIMA(df_developed_day['Confirmed'], order=(1,2,2)).fit()

In [ ]:
arima_model_122.summary()

In [ ]:
start = df_day.index.get_loc('2021-05-13')
end = df_day.index.get_loc('2021-06-01')
print(start,'to',end)

In [ ]:
 predict_122= arima_model_122.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_122=arima_model_122.predict(start=len(df_developed_day)+1,end=len(df_developed_day)+(300),dynamic=False)
print("The length of pred_future values :",len(pred_future_122))

In [ ]:
plt.plot(df_day['Confirmed'],color='blue',label='Original')
plt.plot(predict_122,color='yellow',label='Testing')
plt.plot(pred_future_122,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
 predict_122= arima_model_122.predict(start=len(train) ,end=len(df_developed_day)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
r2=r2_score(test['Confirmed'],predict_122)
mse=mean_squared_error(test['Confirmed'],predict_122)
mae=mean_absolute_error(test['Confirmed'],predict_122)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

****Forecasting the impact of Covid-19 in developing states in India****

The Six most developing states in India are

They are
*  Mizoram
* Tripura
* Punjab
* Odisha
* Assam
*Manipur


So, initially we will be trying to study the impact of covid-19 cases in developed states of India. 

The list was taken based on the data avilable in the link given below.


https://statisticstimes.com/economy/india/indian-states-gdp-growth.php

In [ ]:
df2=pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
df_developing_=df2.drop(columns=['ConfirmedIndianNational','ConfirmedForeignNational','Cured','Deaths','Time','Sno'])

In [ ]:
df_developing_.shape


In [ ]:
Miz=df_developing_[df_developing_['State/UnionTerritory']=='Mizoram']
Tirp=df_developing_[df_developing_['State/UnionTerritory']=='Tripura']
Pj=df_developing_[df_developing_['State/UnionTerritory']=='Punjab']
Od=df_developing_[df_developing_['State/UnionTerritory']=='Odisha']
Asm=df_developing_[df_developing_['State/UnionTerritory']=='Assam']
Mani=df_developing_[df_developing_['State/UnionTerritory']=='Manipur']


In [ ]:


with plt.xkcd():
  fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(8,8), squeeze=False, sharex=True, sharey=False, constrained_layout=True )
  sns.kdeplot(data=Miz['Confirmed'],ax=ax[0,0],fill=True)
  ax[0,0].set_title('Mizoran')
  sns.kdeplot(data=Tirp['Confirmed'],ax=ax[0,1],fill=True)
  ax[0,1].set_title('Tripura')
  sns.kdeplot(data=Pj['Confirmed'],ax=ax[1,0],fill=True)
  ax[1,0].set_title('Punjab')

  ax[1,1].set_title('Odisha')
  sns.kdeplot(data=Od['Confirmed'],ax=ax[1,1],fill=True)
  ax[2,0].set_title('Assam')
  sns.kdeplot(data=Asm['Confirmed'],ax=ax[2,0],fill=True)
  ax[2,1].set_title('Manipur')
  sns.kdeplot(data=Mani['Confirmed'],ax=ax[2,1],fill=True)
  plt.show()

In [ ]:
states=['Mizoram','Tripura','Punjab','Odisha','Assam','Manipur']

In [ ]:
df_developing_=df_developing_[df_developing_['State/UnionTerritory'].isin(states)]

In [ ]:
df_developing_.shape

In [ ]:
df_developing_['State/UnionTerritory'].unique()

In [ ]:
df_developing_.drop(columns=['State/UnionTerritory'],inplace=True)

In [ ]:
df_developing_['Date']=pd.to_datetime(df_developing_['Date'], format='%Y-%m-%d')

In [ ]:
df_developing_.set_index('Date',inplace=True)

In [ ]:
df_developing_=df_developed_.resample('D').sum()

In [ ]:
result = seasonal_decompose(df_developing_['Confirmed'])
result.plot()
plt.show()

In [ ]:
test_stationarity(df_developing_['Confirmed'])

In [ ]:
df_developing_['diff1']=df_developing_['Confirmed']-df_developing_['Confirmed'].shift(1)
test_stationarity(df_developing_['diff1'].dropna())

In [ ]:
df_developing_['diff2']=df_developing_['Confirmed']-df_developing_['Confirmed'].shift(2)
test_stationarity(df_developing_['diff2'].dropna())

In [ ]:
df_developing_['diff3']=df_developing_['Confirmed']-df_developing_['Confirmed'].shift(3)
test_stationarity(df_developing_['diff3'].dropna())

In [ ]:
plot_acf(df_developing_['diff2'].dropna(),lags=15)
plt.show()

In [ ]:
plot_pacf(df_developing_['diff2'].dropna(),lags=15)
plt.show()

In [ ]:
# Identifying Best Suitable model 
auto_arima_model=auto_arima(df_developing_['Confirmed'],trace=True,Supress_warnings=True)

In [ ]:
#Best Model 
arima_model_122 = ARIMA(df_developing_['Confirmed'], order=(1,2,2)).fit()

In [ ]:
arima_model_122.summary()

In [ ]:
start = df_day.index.get_loc('2021-05-13')
end = df_day.index.get_loc('2021-06-01')
print(start,'to',end)

In [ ]:
 predict_122= arima_model_122.predict(start=start+1 ,end=end+1, dynamic= False)  

In [ ]:
#Trying to forecast future values
pred_future_122=arima_model_122.predict(start=len(df_developing_)+1,end=len(df_developing_)+(300),dynamic=False)
print("The length of pred_future values :",len(pred_future_122))

In [ ]:
plt.plot(df_developing_['Confirmed'],color='blue',label='Original')
plt.plot(predict_122,color='green',label='Testing')
plt.plot(pred_future_122,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
train=df_developed_day[:len(df_day)-(89)]
test=df_developed_day[len(df_day)-89:]

In [ ]:
len(df_developing_)

In [ ]:
 predict_122= arima_model_122.predict(start=len(train) ,end=len(df_developing_)-1, dynamic= False)  

In [ ]:
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
rmse=sqrt(mean_squared_error(test['Confirmed'],predict_122))
r2=r2_score(test['Confirmed'],predict_122)
mse=mean_squared_error(test['Confirmed'],predict_122)
mae=mean_absolute_error(test['Confirmed'],predict_122)
print('rmse:',rmse)
print('r2_score:',r2)
print('Mean_absolute_error:',mae)
print('Mean squared error:',mse)

In [ ]:
#Trying to forecast future values
pred_future_500=arima_model_122.predict(start=len(df_developing_)+1,end=len(df_developing_)+(500),dynamic=False)
print("The length of pred_future values :",len(pred_future_500))

In [ ]:
plt.plot(df_developing_['Confirmed'],color='blue',label='Original')
plt.plot(predict_122,color='green',label='Testing')
plt.plot(pred_future_500,color='red',label='Predicted')
plt.xticks(rotation=45)
plt.legend()
plt.show()